In [2]:
import pandas as pd
import numpy as np
import datetime

In [6]:
data = pd.read_csv('data/grided_data.csv')

data['created_at'] = pd.to_datetime(data['created_at'])

data['hour'] = data['created_at'].dt.hour

data['day_of_month'] = data['created_at'].dt.day

data['week_day'] = data['created_at'].dt.weekday

data['month'] = data['created_at'].dt.month

# data.drop(columns=['pickup_lat', 'pickup_lng', 'dropoff_lat', 
#                    'dropoff_lng', 'user_id', 'driver_id', 
#                    'ride_distance', 'canceled_by_client',
#                    'canceled_by_driver', 'ride_to_suburb'
#                   ], inplace=True)

# grouped_data = data.groupby(['index_cell', 'month', 'day_of_month', 'hour', 'week_day']).agg('count')

# grouped_data.reset_index(inplace=True)

# grouped_data.rename(columns={'created_at': 'forecast_volume'}, inplace=True)

# grouped_data.sort_values(by=['index_cell', 'month', 'day_of_month', 'hour'])


# times = pd.date_range('2018-08-21', '2019-02-21 16:00:00', freq='1h')

# a = list(times)
# b = list(range(69))

# index = pd.MultiIndex.from_product([a, b], names = ["created_at", "index_cell"])

# full_df = pd.DataFrame(index = index).reset_index()

# full_df['hour'] = full_df['created_at'].dt.hour

# full_df['day_of_month'] = full_df['created_at'].dt.day

# full_df['week_day'] = full_df['created_at'].dt.weekday

# full_df['month'] = full_df['created_at'].dt.month

# full_df.drop(columns=['created_at'], inplace=True)

# data_with_all_times = pd.merge(full_df, grouped_data, how='left', on=['index_cell', 'month', 'day_of_month', 'hour', 'week_day'])

# data_with_all_times.fillna(0, inplace=True)


In [8]:
data.to_csv('andrew.csv', index=False)

In [3]:
#REad new data
canceled_ratio = pd.read_csv('data/canceled_ratio.csv')
mean_distance_per_area = pd.read_csv('data/mean_distance_per_area.csv')
unique_drivers_per_area = pd.read_csv('data/unique_drivers_per_area.csv', header=None, names = ['index_cell','unique_drivers_per_area'])
unique_users_per_area = pd.read_csv('data/unique_users_per_area.csv',  header=None, names = ['index_cell','unique_users_per_area'])
suburb = pd.read_csv('data/suburb.csv')

In [4]:
data_with_all_times = pd.merge(data_with_all_times, mean_distance_per_area, how='left', on=['index_cell'])
data_with_all_times = pd.merge(data_with_all_times, canceled_ratio, how='left', on=['index_cell'])
data_with_all_times = pd.merge(data_with_all_times, unique_drivers_per_area, how='left', on=['index_cell'])
data_with_all_times = pd.merge(data_with_all_times, unique_users_per_area, how='left', on=['index_cell'])
data_with_all_times = pd.merge(data_with_all_times, suburb, how='left', on=['index_cell'])

In [5]:
drop_zones = data_with_all_times.groupby('index_cell').sum()[data_with_all_times.groupby('index_cell').sum()['forecast_volume']<300]

list_drop_zones = list(drop_zones.index)

list_drop_zones

for i in list_drop_zones:
    data_with_all_times = data_with_all_times[data_with_all_times['index_cell'] != i]


In [6]:
data_with_all_times['top_zones'] = 0

top_zon = [35, 34 ,27 ,36 ,42, 26,25, 18, 28, 41]
for i in top_zon:
    data_with_all_times.loc[data_with_all_times['index_cell'] == i, 'top_zones'] = 1

In [7]:
weather = pd.read_excel('data/weather.xls')

weather = weather[['Местное время во Львове (аэропорт)', 'T','U','DD','Ff','RRR']]

weather.DD = weather.DD.str.split(',').apply(lambda x: x[0])

weather.RRR.fillna(0, inplace=True)

weather.DD[weather.DD == 'Ветер'] = 1

weather.DD[weather.DD == 'Штиль'] = 0

weather.RRR[weather.RRR == "Следы осадков"] = 0

weather.iloc[:,0] = pd.to_datetime(weather.iloc[:,0])

c:\users\user\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
c:\users\user\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\user\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexin

In [8]:
weather['Местное время во Львове (аэропорт)'] = pd.to_datetime(weather['Местное время во Львове (аэропорт)'])

weather['hour'] = weather['Местное время во Львове (аэропорт)'].dt.hour

weather['day_of_month'] = weather['Местное время во Львове (аэропорт)'].dt.day

weather['week_day'] = weather['Местное время во Львове (аэропорт)'].dt.weekday

weather['month'] = weather['Местное время во Львове (аэропорт)'].dt.month

weather = weather.drop(columns=['Местное время во Львове (аэропорт)'])

In [9]:
data_with_all_times = pd.merge(data_with_all_times, weather, how='left', on=['month', 'day_of_month', 'hour', 'week_day']).bfill().ffill()

In [10]:
test_index = data_with_all_times.loc[np.logical_and(data_with_all_times['month'] == 2, 
                                data_with_all_times['day_of_month'] > 14)].sort_values(by=['day_of_month', 'hour']).index

In [15]:
train = data_with_all_times.drop(index=test_index)

test = data_with_all_times.loc[test_index]

In [1559]:
pd.DataFrame(train).to_csv('data/train.csv', index=False)
pd.DataFrame(test).to_csv('data/test.csv', index=False)